# Image Segmentation Example
##### Taken from TensorFlow example
https://www.tensorflow.org/tutorials/images/segmentation

In [1]:
import pandas as pd
import cv2

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import warnings
warnings.filterwarnings('ignore')

2023-10-03 14:44:57.781921: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
file_name = 'project-1-at-2023-09-28-18-34-9bdf5079.csv'

In [4]:
base_df = pd.read_csv(file_name)
base_df

,annotation_id,annotator,created_at,id,image,label,lead_time,updated_at
0,5,1,2023-09-14T02:27:39.290593Z,1,/data/upload/1/1396640e-B1Afghanistan_85.jpg,"[{""x"":1.9917830250057826,""y"":1.285367155491171...",170.006,2023-09-14T02:27:39.290616Z
1,8,1,2023-09-25T20:04:29.798594Z,2,/data/upload/1/7674dfd6-B1Bahamas_206.jpg,"[{""x"":13.838268792710707,""y"":20.72892938496583...",44.938,2023-09-25T20:04:29.798635Z
2,7,1,2023-09-14T02:37:09.432244Z,3,/data/upload/1/a2b61e20-B1Brazil_438.jpg,"[{""x"":59.24854201447221,""y"":6.165703275529865,...",113.256,2023-09-14T02:38:05.846943Z
3,9,1,2023-09-25T20:07:05.459738Z,4,/data/upload/1/c4b7610f-B1China_117.jpg,"[{""x"":0,""y"":1.1389521640091116,""width"":21.1845...",152.334,2023-09-25T20:07:05.459793Z
4,10,1,2023-09-25T20:14:59.542319Z,5,/data/upload/1/bbe6bd0b-B1China_116.jpg,"[{""x"":16.073003159673757,""y"":66.20184100725108...",472.354,2023-09-25T20:14:59.542348Z
5,11,1,2023-09-25T20:17:39.626375Z,6,/data/upload/1/dc5fb95e-B1Thailand_197.jpg,"[{""x"":37.55027234009706,""y"":16.377649325626198...",156.731,2023-09-25T20:17:39.626398Z


In [5]:
for i in range(len(base_df)):
    base_df.image[i] = base_df.image[i].split("/")
    base_df.image[i] = base_df.image[i][4]
    base_df.image[i] = base_df.image[i].split("-")
    base_df.image[i] = base_df.image[i][1]
    base_df.image[i] = 'ARTS-AND-ENTERTAINMENT/'+base_df.image[i]

In [6]:
base_df.image

0    ARTS-AND-ENTERTAINMENT/B1Afghanistan_85.jpg
1       ARTS-AND-ENTERTAINMENT/B1Bahamas_206.jpg
2        ARTS-AND-ENTERTAINMENT/B1Brazil_438.jpg
3         ARTS-AND-ENTERTAINMENT/B1China_117.jpg
4         ARTS-AND-ENTERTAINMENT/B1China_116.jpg
5      ARTS-AND-ENTERTAINMENT/B1Thailand_197.jpg
Name: image, dtype: object

In [7]:
training_df = pd.DataFrame(columns = ['FileName', 'x_min', 'x_max', 'y_min', 'y_max', 'class_name'])

In [8]:
base_df.label[0]

'[{"x":1.9917830250057826,"y":1.2853671554911719,"width":21.909613275063606,"height":5.438091811693419,"rotation":0,"rectanglelabels":["Graphic"],"original_width":996,"original_height":1672},{"x":3.1699427693159454,"y":8.804672252407542,"width":2.4364806671287904,"height":1.374539637823726,"rotation":0,"rectanglelabels":["Button"],"original_width":996,"original_height":1672},{"x":1.659819187504819,"y":11.469430002844303,"width":22.407559031315056,"height":24.619724747484753,"rotation":0,"rectanglelabels":["Menu"],"original_width":996,"original_height":1672},{"x":25.73487313372787,"y":-1.2846e-320,"width":74.26512686627214,"height":58.71774107762258,"rotation":0,"rectanglelabels":["Picture"],"original_width":996,"original_height":1672},{"x":25.486381794215262,"y":58.79335574857135,"width":74.4105564931903,"height":20.628122302635795,"rotation":0,"rectanglelabels":["Text"],"original_width":996,"original_height":1672},{"x":62.69166004081011,"y":80.46516281056655,"width":15.871503739544703

In [9]:
boxes = base_df.label
for i in range(0, len(boxes)):
     boxes.iloc[i] = boxes.iloc[i].split(",")


In [10]:
for i in range(0, len(boxes)):
    for j in range(0, len(boxes[i])):
        boxes[i][j] = boxes[i][j].replace('"', '')
        boxes[i][j] = boxes[i][j].replace('[', '')
        boxes[i][j] = boxes[i][j].replace(']', '')
        boxes[i][j] = boxes[i][j].replace('{', '')
        boxes[i][j] = boxes[i][j].replace('}', '')
        boxes[i][j] = boxes[i][j].split(":")

In [11]:
temp_df = pd.DataFrame(columns = ['image', 'x', 'y', 'width', 'height', 'rotation', 'rectanglelabels',
                       'original_width', 'original_height'])
for i in range(len(boxes)):
    for j in range(0, int(len(boxes[i])/8), 8):
        temp_df = temp_df.append({'image' : base_df.image[i], 'x' : boxes[i][j*8][1], 'y' : boxes[i][j*8+1][1], 'width' : boxes[i][j*8+2][1],
                        'height' : boxes[i][j*8+3][1], 'rotation' : boxes[i][j*8+4][1], 
                        'rectanglelabels' : boxes[i][j*8+5][1], 'original_width' : boxes[i][j*8+6][1],
                        'original_height' : boxes[i][j*8+7][1]}, ignore_index = True)


In [12]:
def normalize(xmin, ymin, width, height, og_w, og_h):
    xmax = xmin + width
    ymax = ymin + height
    return xmin/og_w, xmax/og_w, ymin/og_h, ymax/og_h

In [13]:
for i in range(len(temp_df)):
    x_max = str(float(temp_df.x[i]) + float(temp_df.width[i]))
    y_max = str(float(temp_df.y[i]) + float(temp_df.height[i]))
    training_df = training_df.append({'FileName': temp_df.image[i], 'x_min': temp_df.x[i], 'x_max': x_max , 'y_min': temp_df.y[i], 'y_max': y_max, 'class_name': temp_df.rectanglelabels[i]}, ignore_index = True)

In [14]:
training_df.to_csv('train.csv')
train_df = pd.read_csv('train.csv')
# for training
with open("train.txt", "w+") as f:
  for idx, row in train_df.iterrows():
      x1 = row['x_min']
      x2 = row['x_max'] 
      y1 = row['y_min']
      y2 = row['y_max'] 
      
      fileName = row['FileName']
      className = row['class_name']
      f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + className + '\n')

In [15]:
f.close()